In [1]:
# Import Library
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from tensorflow.keras.layers import Conv2D,MaxPooling2D,InputLayer
import os
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

In [2]:
# Variable Definition
num_classes=5
height,width=48,48
batch_size=32
input_shape = (height,width,3)

In [3]:
data = pd.read_csv("icml_face_data.csv")
data.columns = ['emotion', 'Usage', 'pixels']

In [5]:
data['Usage'].value_counts()

Training       28709
PublicTest      3589
PrivateTest     3589
Name: Usage, dtype: int64

In [6]:
def prepare_data(data):
    image_array = np.zeros(shape=(len(data), 48, 48, 3))
    image_label = np.array(list(map(int, data['emotion'])))

    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, 'pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48)) 
        image_array[i, :, :, 0] = image / 255

    return image_array, image_label

In [7]:
full_train_images, full_train_labels = prepare_data(data[data['Usage']=='Training'])
test_images, test_labels = prepare_data(data[data['Usage']!='Training'])

print(full_train_images.shape)
print(full_train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

(28709, 48, 48, 3)
(28709,)
(7178, 48, 48, 3)
(7178,)


In [8]:
train_images, valid_images, train_labels, valid_labels =\
    train_test_split(full_train_images, full_train_labels, test_size=0.2, random_state=1)

print(train_images.shape)
print(valid_images.shape)
print(train_labels.shape)
print(valid_labels.shape)

(22967, 48, 48, 3)
(5742, 48, 48, 3)
(22967,)
(5742,)


In [9]:

resnet = ResNet50(include_top=False, weights='imagenet', input_shape= (height, width,3))

for layer in resnet.layers:
    layer.trainable = True
#x = keras.layers.Flatten()(resnet.output)
#x = keras.layers.Dense(100, activation='relu')(x)
#x = keras.layers.Dense(100, activation='relu')(x)
#x = keras.layers.Dense(100, activation='relu')(x)

resnet.summary()

94773248/94765736 [==============================] - 8s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 48, 48, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 54, 54, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 24, 24, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 24, 24, 64)   256         conv1_conv[0][0]                 
__________________________

In [10]:
model = Sequential()
model.add(resnet)#, input_dim=input_shape)
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(48,48,3)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(5, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2, 2, 2048)        23587712  
_________________________________________________________________
conv2d (Conv2D)              (None, 2, 2, 128)         2359424   
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 2, 2, 128)         147584    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 1, 1, 128)         0         
_________________________________________________________________
dropout (Dropout)            (None, 1, 1, 128)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1, 1, 128)         512       
_________________________________________________________________
flatten (Flatten)            (None, 128)               0

In [12]:
opt = keras.optimizers.Adam(lr=0.001)
model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=opt, metrics=['accuracy'])

In [13]:
h1 = model.fit(train_images, train_labels, batch_size=256, epochs=2, verbose=1, 
                   validation_data =(valid_images, valid_labels))

Epoch 1/2
90/90 [==============================] - 44s 213ms/step - loss: nan - accuracy: 0.1394 - val_loss: nan - val_accuracy: 0.1371
Epoch 2/2
90/90 [==============================] - 14s 161ms/step - loss: nan - accuracy: 0.1397 - val_loss: nan - val_accuracy: 0.1371
